In [231]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import pandas as pd
from sklearn import svm
import numpy as np
import math

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors
from sklearn.model_selection import train_test_split
from sklearn import datasets
from matplotlib.colors import ListedColormap
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV, LeaveOneOut, cross_val_predict
from sklearn.model_selection import KFold  
from sklearn.linear_model import LogisticRegression  
from sklearn.naive_bayes import GaussianNB  
from sklearn.neighbors import KNeighborsClassifier   
from sklearn import svm  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
import math
import datetime
import multiprocessing as mp
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

import random


# In[25]:


# cores = mp.cpu_count()
# pool = mp.Pool(processes=20)

# min1=min(list(X_standard.shape))
# a1=list(range(10,min1,10))
# a1[-1]=min1
# pcc_top1=pool.map(find_best_pca_lr,a1)  ##（SVM，LR）
# pool.terminate()


# a1=pd.DataFrame()
# for i1 in pcc_top1:
#     a1=pd.concat([a1,i1], axis=0)
# a1.columns=['tr_TP', 'tr_TN', 'tr_FP', 'tr_FN', 'tr_Sen', 'tr_Spe', 'tr_Acc', 'tr_PPV', 'tr_NPV', 'tr_MCC', 'tr_AUC','TP',
#        'TN', 'FP', 'FN', 'Sen', 'Spe', 'Acc', 'PPV', 'NPV', 'MCC', 'AUC','PCA']   

# a1=a1.sort_values(['Acc','tr_Acc','PCA'], ascending=[False,False,True])

In [232]:
def read_single_csv(input_path,i):

    df_chunk=pd.read_csv(input_path,
                         chunksize=10000,
#                          sep='\t'
                        sep=i
                        )
    res_chunk=[]
    for chunk in df_chunk:
        res_chunk.append(chunk)
    res_df=pd.concat(res_chunk)
    return res_df

In [233]:
def get_measures_gridloo(label, score):
    label = np.array(label)
    score = np.array(score)
    
    N  = len(label)
    TP = sum((label == 1) & (score == 1))
    TN = sum((label == 0) & (score == 0))
    FP = sum((label == 0) & (score == 1))
    FN = sum((label == 1) & (score == 0))

    # init all measures to nan
    measures = {measure: float("nan") for measure in ("Sen", "Spe", "Acc", "PPV", "NPV", "MCC","AUC")}
    
    measures["TP"] = TP
    measures["TN"] = TN
    measures["FP"] = FP
    measures["FN"] = FN
    
    S = (TP + FN) / N
    P = (TP + FP) / N

    if (TP + FN) > 0:
        measures["Sen"] = round(TP/(TP+FN), 4)

    if (TN + FP) > 0:
        measures["Spe"] = round(TN/(TN+FP), 4)

    if (TP + FP + FN + TN) > 0:
        measures["Acc"] = round((TP+TN)/(TP+FP+FN+TN), 4)

    if (TP + FP) > 0:
        measures["PPV"] = round(TP/(TP+FP), 4)

    if (TN + FN) > 0:
        measures["NPV"] = round(TN/(TN+FN), 4)

    if (S*P*(1-S)*(1-P)) > 0:
        measures["MCC"] = round((TP/N - S*P)/(math.sqrt(S*P*(1-S)*(1-P))), 4)
    
    
    measures["AUC"]= roc_auc_score(label, score)
    return pd.DataFrame([measures],
                        columns=["TP", "TN", "FP",
                                 "FN", "Sen", "Spe", "Acc", "PPV", "NPV", "MCC","AUC"])

In [234]:
##SVM
def find_best_pca(i):
    n_components=i
    estimator = PCA(n_components=n_components,random_state=10)
    pca_X_train = estimator.fit_transform(X_standard)
    pca_X_test = estimator.transform(X_standard_test)
    
    scoring = {'AUC': 'roc_auc'}
    cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
    gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
    parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]
    
    cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 
    grid_search=GridSearchCV(estimator=SVC(random_state=10),param_grid=parameters,cv=cvx,scoring='accuracy')
    grid_search.fit(pca_X_train, train_y)
    
    train_pre_y = cross_val_predict(grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
    train_res1=get_measures_gridloo(train_y,train_pre_y)
#     train_res1['PCA']=n_components
    test_pre_y = grid_search.predict(pca_X_test)
    test_res1=get_measures_gridloo(test_y,test_pre_y)
    test_res1['PCA']=n_components
    
    res=pd.concat([train_res1,test_res1], axis=1)
    return res

In [235]:
def find_best_pca_lr(i):
    n_components=i
    estimator = PCA(n_components=n_components,random_state=10)
    pca_X_train = estimator.fit_transform(X_standard)
    pca_X_test = estimator.transform(X_standard_test)
    
    cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 

    param_grid = {'penalty':['l2'],
              "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],  
              "solver":["newton-cg", "lbfgs","liblinear"],
#             "C":list(np.arange(0,0.1,0.005)),   
#               "solver":["newton-cg", "lbfgs","liblinear"]
            }
    LR_grid = LogisticRegression()
    grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring='accuracy',n_jobs=9)
    grid_search.fit(pca_X_train, train_y)


    train_pre_y = cross_val_predict(grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
    train_res1=get_measures_gridloo(train_y,train_pre_y)
#     train_res1['PCA']=n_components
   

    test_pre_y = grid_search.predict(pca_X_test)
    test_res1=get_measures_gridloo(test_y,test_pre_y)
    test_res1['PCA']=n_components
    
    res=pd.concat([train_res1,test_res1], axis=1)
    return res

In [268]:
cancer='BRCA'
path='D:\\lymph_meta_xmiseq\\tcga_data\\'
# e1="./"+cancer+"_delta_pcc_wilcox.csv"
# delta_pcc=read_single_csv(e1,',')
# delta_pcc=delta_pcc.drop(['Unnamed: 0','pair'],axis=1)
# delta_pcc=delta_pcc.dropna(axis=0,how='any')

# result=delta_pcc.T
# train=result.iloc[:,0:-1].values.astype('float')
# target=result.iloc[:,-1].values.astype('float')

train=pd.read_csv(path+cancer+"\\admat_final.csv").iloc[:,2:].T.values
target=pd.read_csv(path+cancer+"\\gctm_label.csv",index_col=0).values.ravel()

In [263]:
train

array([[-1.19461937e-04, -8.63151370e-03,  3.10418355e-03, ...,
        -1.52881284e-02,  9.97735338e-04, -5.38560918e-03],
       [-8.24605516e-04, -1.42506638e-03,  7.95171455e-03, ...,
         3.31465816e-04,  2.26944011e-03, -9.78459003e-05],
       [-1.16900878e-02, -5.33698509e-03,  6.28624170e-03, ...,
         2.71030467e-02, -7.49026964e-03, -2.40547544e-03],
       ...,
       [ 3.34385978e-02,  2.44368202e-02,  4.33853074e-02, ...,
        -4.67998447e-04, -1.14356119e-02, -2.56179085e-03],
       [ 4.88828589e-04, -3.34588148e-03, -7.58491028e-03, ...,
        -3.99659676e-03, -3.38371248e-03,  4.06080068e-03],
       [-5.09754182e-03, -7.55936433e-02, -3.46494864e-02, ...,
         3.19405065e-03,  1.86598804e-03,  7.88591094e-04]])

In [264]:
target

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [183]:
train_X,test_X, train_y, test_y = train_test_split(train,
                                                   target,
                                                   test_size = 0.3,
                                                   stratify=target,
                                                   random_state=10
                                                   )

standardScaler = StandardScaler()
standardScaler.fit(train_X)
X_standard = standardScaler.transform(train_X)
X_standard_test = standardScaler.transform(test_X)

n_components=0.95
estimator = PCA(n_components=n_components,random_state=10)
pca_X_train = estimator.fit_transform(X_standard)
pca_X_test = estimator.transform(X_standard_test)
cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 

In [184]:
train_X.shape

(126, 2281)

In [185]:
#svm
scoring = {'AUC': 'roc_auc'}
cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]
cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 

svc_grid_search=GridSearchCV(estimator=SVC(random_state=10),param_grid=parameters,cv=cvx,scoring='accuracy')
svc_grid_search.fit(pca_X_train, train_y)

#LR
param_grid = {'penalty':['l1', 'l2'],
              "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],
              "solver":["newton-cg", "lbfgs","liblinear","sag","saga"]
#               "algorithm":['auto', 'ball_tree', 'kd_tree', 'brute']
            }
LR_grid = LogisticRegression(max_iter=1000)
LR_grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring='accuracy',n_jobs=10)
LR_grid_search.fit(pca_X_train, train_y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.45230769        nan 0.52369231 0.54769231
 0.54769231 0.72153846 0.76123077 0.74523077        nan        nan
 0.45230769        nan 0.50030769 0.75292308 0.75292308 0.75292308
 0.79230769 0.75292308        nan        nan 0.45230769        nan
 0.52369231 0.848      0.848      0.82430769 0.864      0.832
        nan        nan 0.68246154        nan 0.75292308 0.888
 0.888      0.87230769 0.88       0.88              nan        nan
 0.92              nan 0.92       0.88061538 0.88061538 0.85661538
 0.86461538 0.86461538        nan        nan 0.944             nan
 0.936      0.84061538 0.84061538 0.86461538 0.86461538 0.87261538
        nan        nan 0.944             nan 0.87261538 0.84861538
 0.84861538 0.86461538 0.86461538 0.87261538        nan        nan
 0.90430769        nan 0.87261538 0.85661538 0.85661538 0.84861

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=10, shuffle=True),
             estimator=LogisticRegression(max_iter=1000), n_jobs=10,
             param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100,
                               1000],
                         'penalty': ['l1', 'l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='accuracy')

In [186]:
estimators = [
    ('lr', LR_grid_search.best_estimator_),
    ('svc', svc_grid_search.best_estimator_),
]
clf = StackingClassifier(estimators=estimators, final_estimator=LinearSVC(C=5),n_jobs=10)
clf.fit(pca_X_train, train_y)

StackingClassifier(estimators=[('lr',
                                LogisticRegression(C=1, max_iter=1000,
                                                   penalty='l1',
                                                   solver='liblinear')),
                               ('svc',
                                SVC(C=32, gamma=3.0517578125e-05,
                                    random_state=10))],
                   final_estimator=LinearSVC(C=5), n_jobs=10)

In [187]:
estimators = [
    ('lr', LR_grid_search.best_estimator_),
    ('svc', svc_grid_search.best_estimator_),
]

param_grid = {'final_estimator':[LogisticRegression(C=0.00001),LogisticRegression(C=0.0001),LogisticRegression(C=0.001),LogisticRegression(C=0.01),
                                 LogisticRegression(C=0.1),LogisticRegression(C=1),LogisticRegression(C=10),LogisticRegression(C=100),
                                 LogisticRegression(C=1000)]}

Stacking_grid =StackingClassifier(estimators=estimators,)
Stacking_grid_search = GridSearchCV(Stacking_grid, param_grid=param_grid, cv=cvx ,scoring='accuracy',n_jobs=10)
Stacking_grid_search.fit(pca_X_train, train_y)
Stacking_grid_search.best_estimator_

StackingClassifier(estimators=[('lr',
                                LogisticRegression(C=1, max_iter=1000,
                                                   penalty='l1',
                                                   solver='liblinear')),
                               ('svc',
                                SVC(C=32, gamma=3.0517578125e-05,
                                    random_state=10))],
                   final_estimator=LogisticRegression(C=100))

In [188]:
train_pre_y = cross_val_predict(Stacking_grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
train_res1=get_measures_gridloo(train_y,train_pre_y)
#     train_res1['PCA']=n_components

test_pre_y = Stacking_grid_search.predict(pca_X_test)
test_res1=get_measures_gridloo(test_y,test_pre_y)

In [189]:
print(train_res1)
print(test_res1)

   TP  TN  FP  FN     Sen     Spe     Acc     PPV     NPV     MCC       AUC
0  64  50   7   5  0.9275  0.8772  0.9048  0.9014  0.9091  0.8076  0.902365
   TP  TN  FP  FN  Sen  Spe  Acc  PPV  NPV  MCC  AUC
0  30  24   0   0  1.0  1.0  1.0  1.0  1.0  1.0  1.0


In [ ]:
%%time
################random seed loop#################
train_scores=[]
test_scores=[]
loopn=20
scoring='roc_auc'
np.random.seed(42)
random_states=np.random.randint(1,101,loopn)
for i in range(loopn):
    train_X,test_X, train_y, test_y = train_test_split(train,
                                                   target,
                                                   test_size = 0.3,
                                                   stratify=target,
                                                   random_state=random_states[i]
                                                   )

    standardScaler = StandardScaler()
    standardScaler.fit(train_X)
    X_standard = standardScaler.transform(train_X)
    X_standard_test = standardScaler.transform(test_X)

    #n_components=0.95
    #n_components=range(10,min(train_X.shape),10)
    n_components=[0.99,0.95,0.90,0.85]
    best_pca_train_scores=[]
    best_pca_test_scores=[]
    for j in n_components:
        estimator = PCA(n_components=j,random_state=random_states[i])
        pca_X_train = estimator.fit_transform(X_standard)
        pca_X_test = estimator.transform(X_standard_test)
        cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_states[i]) 
    
        #scoring = {'AUC': 'roc_auc'}
        cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
        gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
        parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]
        cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_states[i]) 

        svc_grid_search=GridSearchCV(estimator=SVC(random_state=random_states[i]),param_grid=parameters,cv=cvx,scoring='accuracy')
        svc_grid_search.fit(pca_X_train, train_y)

        param_grid = {'penalty':['l1', 'l2'],
                      "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],
                      "solver":["newton-cg", "lbfgs","liblinear","sag","saga"]
    #               "algorithm":['auto', 'ball_tree', 'kd_tree', 'brute']
                    }
        LR_grid = LogisticRegression(max_iter=1000)
        LR_grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring=scoring,n_jobs=10)
        LR_grid_search.fit(pca_X_train, train_y)

        estimators = [
            ('lr', LR_grid_search.best_estimator_),
            ('svc', svc_grid_search.best_estimator_),
        ]
        clf = StackingClassifier(estimators=estimators, final_estimator=LinearSVC(C=5),n_jobs=10)
        clf.fit(pca_X_train, train_y)

        estimators = [
            ('lr', LR_grid_search.best_estimator_),
            ('svc', svc_grid_search.best_estimator_),
        ]

        param_grid = {'final_estimator':[LogisticRegression(C=0.00001),LogisticRegression(C=0.0001),LogisticRegression(C=0.001),LogisticRegression(C=0.01),
                                         LogisticRegression(C=0.1),LogisticRegression(C=1),LogisticRegression(C=10),LogisticRegression(C=100),
                                         LogisticRegression(C=1000)]}

        Stacking_grid =StackingClassifier(estimators=estimators,)
        Stacking_grid_search = GridSearchCV(Stacking_grid, param_grid=param_grid, cv=cvx ,scoring=scoring,n_jobs=10)
        Stacking_grid_search.fit(pca_X_train, train_y)
        Stacking_grid_search.best_estimator_

        train_pre_y = cross_val_predict(Stacking_grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
        train_res1=get_measures_gridloo(train_y,train_pre_y)

        test_pre_y = Stacking_grid_search.predict(pca_X_test)
        test_res1=get_measures_gridloo(test_y,test_pre_y)
        
        best_pca_train_scores.append(train_res1.loc[:,"AUC"])
        best_pca_test_scores.append(test_res1.loc[:,"AUC"])
    
    train_scores.append(np.max(best_pca_train_scores))
    test_scores.append(best_pca_test_scores[np.argmax(best_pca_train_scores)])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.75113775
 0.75113775 0.75099266 0.75094456 0.75099266        nan        nan
 0.5               nan 0.5        0.76054612 0.76054612 0.75899383
 0.75909149 0.75885101        nan        nan 0.71155659        nan
 0.71155659 0.7898724  0.7898724  0.77735868 0.77870738 0.77696884
        nan        nan 0.76196468        nan 0.76461978 0.78720643
 0.78720643 0.75664736 0.77557727 0.77929138        nan        nan
 0.74885025        nan 0.77659204 0.76060847 0.76060847 0.72502347
 0.77146474 0.77963192        nan        nan 0.72060296        nan
 0.78002262 0.74239508 0.74239508 0.71255053 0.77141904 0.77953431
        nan        nan 0.71089061        nan 0.77938846 0.73759789
 0.73754979 0.70796271 0.7713702  0.77953577        nan        nan
 0.67686679        nan 0.77948619 0.73532912 0.735186

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.72953123
 0.72953123 0.73006322 0.73001512 0.73001515        nan        nan
 0.5               nan 0.5        0.74165244 0.74165244 0.74126473
 0.74102273 0.74097162        nan        nan 0.69233373        nan
 0.69233373 0.77726771 0.77726771 0.76517885 0.76797693 0.76617727
        nan        nan 0.75715703        nan 0.76126242 0.79486088
 0.79490898 0.76340502 0.76908615 0.7654971         nan        nan
 0.79290802        nan 0.77627837 0.78683159 0.78688043 0.76418498
 0.76142924 0.76315046        nan        nan 0.78508535        nan
 0.76436908 0.77680014 0.77680014 0.76912654 0.76006246 0.7630032
        nan        nan 0.77359546        nan 0.76324814 0.77272679
 0.77267869 0.77224102 0.76025558 0.76285668        nan        nan
 0.77229243        nan 0.76295508 0.77195578 0.7719557

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.73991761
 0.73991761 0.74016111 0.74016111 0.74016111        nan        nan
 0.5               nan 0.5        0.74915233 0.74915233 0.74732018
 0.74698131 0.74669206        nan        nan 0.76860508        nan
 0.76860508 0.77984435 0.77984435 0.76645461 0.77042381 0.76645174
        nan        nan 0.7293695         nan 0.73429961 0.78507775
 0.78512659 0.75875659 0.76427302 0.76658582        nan        nan
 0.77275962        nan 0.76049335 0.77868775 0.77863965 0.74702996
 0.75997644 0.76542173        nan        nan 0.77889467        nan
 0.76518373 0.77242343 0.77242343 0.762287

In [266]:
print('mean')
print(np.mean(train_scores))
print(np.mean(test_scores))
print("std")
print(np.std(train_scores))
print(np.std(test_scores))

0.7559758835597881
0    0.750783
Name: AUC, dtype: float64


In [267]:
print(train_scores)
print(test_scores)

[0.7823665466651183, 0.7603211009174312, 0.7589517284093988, 0.7508806565246003, 0.7323723531916541, 0.7714502767777649, 0.7680195873495143, 0.7460805945883173, 0.7417595710912398, 0.7417595710912398, 0.7823133201718732, 0.7586323694499284, 0.7669695738009522, 0.7325320326713893, 0.739964386637247, 0.7520371230596524, 0.7461338210815622, 0.7823133201718732, 0.7476096465760849, 0.7570500909689156]
[0    0.714706
Name: AUC, dtype: float64, 0    0.763655
Name: AUC, dtype: float64, 0    0.763971
Name: AUC, dtype: float64, 0    0.764811
Name: AUC, dtype: float64, 0    0.792752
Name: AUC, dtype: float64, 0    0.72479
Name: AUC, dtype: float64, 0    0.771744
Name: AUC, dtype: float64, 0    0.738866
Name: AUC, dtype: float64, 0    0.75042
Name: AUC, dtype: float64, 0    0.75042
Name: AUC, dtype: float64, 0    0.739601
Name: AUC, dtype: float64, 0    0.749475
Name: AUC, dtype: float64, 0    0.746429
Name: AUC, dtype: float64, 0    0.801155
Name: AUC, dtype: float64, 0    0.753782
Name: AUC, dty